In [566]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import pickle

In [567]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [568]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [569]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=True, coord=True, outlier=True)
df_census = utils.load_census()

smooth_outlier: max_scale=40


/tmp/working/code-analysis/../modules/utils.py:194: RuntimeWarning: divide by zero encountered in double_scalars
  var[:i] *= (var[i]/var[i-1])
/tmp/working/code-analysis/../modules/utils.py:194: RuntimeWarning: invalid value encountered in multiply
  var[:i] *= (var[i]/var[i-1])
/tmp/working/code-analysis/../modules/utils.py:192: RuntimeWarning: invalid value encountered in double_scalars
  difa = abs(var[i]-var[i-1])


# of fixed cfips: 552
# of fixed value: 903


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  """


In [570]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 7,
        "USE_TREND": False,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'ensemble'
    }

In [571]:
# params={
#         "ensemble": False,
#         "act_thre": 2,
#         "USE_LAG": 5,
#         "USE_OLD_LOG": False,
#         "USE_TREND": False,
#         "blacklist": [],
#         "blacklistcfips": [],
#         "clip": (None, None)
#     }

## バリデーション

### 前の予測結果を用いて予測していく

In [572]:
# accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, params=params)
# accum_pred.accum_validation(m_len=5)

## Submissionの作成

### 予測結果を活用して予測

In [573]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, params=params)
test_subm.create_submission(accum=True, target_scale=[41])

add lag features: max_scale=40
created df_all_dict[40]
valid_times:  41
pred_m:  1
train_times:  40
use df_all_dict[40]
['scale', 'state_i', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc_2021', 'lng', 'lat', 'select_rate1_lag1', 'select_rate1_lag2', 'select_rate1_lag3', 'select_rate1_lag4', 'select_rate1_lag5', 'select_rate1_lag6', 'select_rate1_lag7', 'select_rate1_rsum2', 'select_rate1_rsum4', 'select_rate1_rsum6', 'select_rate1_rsum8', 'select_rate1_rsum10', 'select_active_lag1_diff1', 'select_active_lag1_diff2', 'selec

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
saved output_dic[1].
saved test_subm_accum_2023-02-24_22:46:22


In [574]:
# df_all40 = test_subm.df_all_dict[40]
# df_all40target = df_all40[df_all40['scale']==2]
# df_all40target[['select_rate1_lag1', 'select_rate1_lag2', 'select_rate1_lag3', 'select_rate1_lag4', 'select_rate1_lag5', 'select_rate1_lag6', 'select_rate1_lag7', 'select_rate1_rsum2', 'select_rate1_rsum4', 'select_rate1_rsum6', 'select_rate1_rsum8', 'select_rate1_rsum10', 'select_active_lag1_diff1', 'select_active_lag1_diff2', 'select_active_lag1_diff3', 'select_active_lag1_diff4', 'select_active_lag1_diff5', 'select_active_lag1_diff6', 'select_active_lag1_diff7']]

In [575]:
df_submission= test_subm.df_submission
df_submission = df_submission.reset_index()
df_submission['cfips'] = df_submission['row_id'].apply(lambda x: int(x.split('_')[0]))
df_submission['month'] = df_submission['row_id'].apply(lambda x: x.split('_')[1])
df_subm202301 = df_submission[df_submission['month']=='2023-01-01']

In [576]:
# df_baseline = pd.read_csv('../submission/submission_13992.csv')
df_baseline = pd.read_csv('../submission/submission_13973.csv')
# df_baseline = pd.read_csv('../submission/submission_14308.csv')
df_baseline['cfips'] = df_baseline['row_id'].apply(lambda x: int(x.split('_')[0]))
df_baseline['month'] = df_baseline['row_id'].apply(lambda x: x.split('_')[1])
df_baseline202301 = df_baseline[df_baseline['month']=='2023-01-01'].copy()
df_baseline202301.rename(columns={mbd: 'baseline'}, inplace=True)

In [577]:
df_merged = df_subm202301.merge(df_baseline202301[['row_id', 'baseline']], how='left', on='row_id')
df_merged['smape'] = utils.smape_arr(df_merged[mbd], df_merged['baseline'])

In [578]:
df_merged[df_merged['smape']>=0.08]

,row_id,microbusiness_density,cfips,month,baseline,smape
1038,21095_2023-01-01,0.687887,21095,2023-01-01,0.688482,0.086558
1760,32510_2023-01-01,10.424712,32510,2023-01-01,10.433808,0.087216
1792,35001_2023-01-01,6.231929,35001,2023-01-01,6.226243,0.091281
1809,35031_2023-01-01,1.143672,35031,2023-01-01,1.142757,0.080019
2026,38079_2023-01-01,1.854692,38079,2023-01-01,1.852889,0.097253
2531,48025_2023-01-01,1.164205,48025,2023-01-01,1.163255,0.081714
2646,48255_2023-01-01,2.936382,48255,2023-01-01,2.933919,0.083914
2751,48465_2023-01-01,1.951516,48465,2023-01-01,1.953301,0.091414


In [579]:
df_merged['smape'].mean()

0.01261112205424169